In [1]:
from albumentations import Compose, HorizontalFlip, VerticalFlip, RandomBrightnessContrast, Resize
from albumentations.pytorch.transforms import ToTensorV2


In [ ]:
import os
from PIL import Image
import numpy as np

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        # Check for specific image file extensions if needed e.g., filename.endswith('.jpg') or '.png'
        if filename.endswith(('.png', '.jpg', '.jpeg')):  # Supports multiple image formats
            img_path = os.path.join(folder, filename)
            with Image.open(img_path) as img:
                img_array = np.array(img)  # Convert the image to a numpy array
                images.append(img_array)
        else:
            continue
    return images

In [2]:
def get_augmentation_pipeline():
    return Compose([
        HorizontalFlip(p=0.5),  # Flip the image horizontally with a probability of 0.5
        VerticalFlip(p=0.5),  # Flip the image vertically with a probability of 0.5
        RandomBrightnessContrast(p=0.2),  # Randomly change brightness and contrast
        Resize(height=512, width=512),  # Resize images to 512x512 for uniformity
        ToTensorV2()  # Convert the image to a PyTorch tensor
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [ ]:
import cv2
import json
from albumentations import BboxParams

# Load your JSON annotations
with open('your_annotations.json', 'r') as f:
    annotations = json.load(f)

def augment_image(image_path, annotations, augmentation_pipeline):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    bboxes = [ann['bbox'] for ann in annotations]
    labels = [ann['caption'] for ann in annotations]

    augmented = augmentation_pipeline(image=image, bboxes=bboxes, labels=labels)
    
    augmented_image = augmented['image']
    augmented_bboxes = augmented['bboxes']
    augmented_labels = augmented['labels']
    
    return augmented_image, augmented_bboxes, augmented_labels

In [ ]:
augmentation_pipeline = get_augmentation_pipeline()
image_path = 'path_to_your_image/image_0.jpg'
augmented_image, augmented_bboxes, augmented_labels = augment_image(image_path, annotations[0]['annotations'], augmentation_pipeline)